In [2]:
import math
import json
import pandas as pd
import numpy as np
from pylab import *
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from matplotlib.collections import PatchCollection

In [19]:
def calc_cost(distance):
    """
    determine the cost of the movement according to the rules:
    Each swing costs the particle $D^-2, where D is the straight-line distance of the length of rope used for that particular swing.
    """
    return round(pow(distance, -2),4)

def calc_distance(x1,y1,x2,y2): 
    """
    determine the distance between 2 points on a coordinate plane
    """
    return round(math.sqrt((x2 - x1)**2 + (y2 - y1)**2),4)

def calc_slope(x1,y1,x2,y2):
    ##(0,0) => (5,0)
    if x1 == x2 and y1 == y2:
        return "NaL"
    elif x1 == x2:
        if y1 < y2:
            return "vertical_pos"
        elif y1 > y2:
            return "vertical_neg"
    elif y1 == y2:
        if x1 < x2:
            return "horizontal_pos"
        elif x1 > x2:
            return "horizontal_neg"
    else:
#         if slope > 0:
#             return str(slope)+"_pos"
#         elif slope < 0:
#             return str(slope)+"_neg"
        slope = round((y2-y1)/(x2-x1),4)
    
    return str(slope)+"_"+quandrant_loc(x1,y1,x2,y2)
    
def plot_path(x1,y1,x2,y2, color="b"):
    # plot points
    ax.plot(x1, y1, color+'o')
    ax.plot(x2, y2, color+'o')
    # plot line
    ax.plot([x1, x2],[y1, y2], color+'-')

def lattice_intercepts(origin, radius):
    x = origin[0]
    y = origin[1]
    r_2 = round(pow(radius,2),2) # rounding to nearest 10th...
    xy_2 = (pow(x,2) + pow(y,2))
    lattice_intercepts = []
    i = 0
    while i <= xy_2:
        j = 0
        while j <= xy_2:
            if (pow((i - x),2) + pow((j - y),2) == r_2):
                lattice_intercepts.append((i,j))
            j = j + 1
        i = i + 1
    return lattice_intercepts

def calc_angle(p0, p1, p2):
    """
    calculate the angle (in degrees) for vertex p0 p1 p2 
    """
    v0 = np.array(p0) - np.array(p1)
    v1 = np.array(p2) - np.array(p1)
    angle = np.math.atan2(np.linalg.det([v0,v1]),np.dot(v0,v1))
    return round(np.degrees(angle),4)

def inbounds(origin, thru_pt, distance, slope):

    x = origin[0]
    y = origin[1]
    x_tp = thru_pt[0]
    y_tp = thru_pt[1]
    
    inbounds = True

    if slope == "vertical_pos":
        x1 = x
        y1 = y + distance
        if y1 > 20.5:
            inbounds = False
    elif slope == "vertical_neg":
        x1 = x
        y1 = y - distance
        if y1 < 0.5:
            inbounds = False
    elif slope == "horizontal_pos":
        x1 = x + distance
        y1 = y
        if x1 > 20.5:
            inbounds = False
    elif slope == "horizontal_neg":
        x1 = x - distance
        y1 = y
        if x1 < 0.5:
            inbounds = False
    else:
        slope = int(slope.split("_")[1])
        c = 1/math.sqrt(1+pow(slope,2))
        s = slope/math.sqrt(1+pow(slope,2))

        # QUADRANT 1
        if x < x_tp and y < y_tp:
            x1 = (x+distance*c)
            y1 = (y+distance*s)
        # QUADRANT 2
        elif x < x_tp and y > y_tp:
            x1 = (x+distance*c)
            y1 = (y+distance*s)
        # QUADRANT 3
        elif x > x_tp and y > y_tp:
            x1 = (x-distance*c)
            y1 = (y-distance*s)
        # QUADRANT 4
        elif x > x_tp and y < y_tp:
            x1 = (x-distance*c)
            y1 = (y-distance*s)

        if x1 < 0.5 or x1 > 20.5 or y1 < 0.5 or y1 > 20.5:
            inbounds = False
#     print(str((x,y,x1,y1)))
#     plot_path(x,y,x1,y1,"y")
#     plot_path(x,y,x_tp,y_tp,"c")
    return inbounds

def quandrant_loc(x,y,x1,y1):
    # QUADRANT 1
    if x < x1 and y < y1:
        return "q1"
    # QUADRANT 2
    elif x < x1 and y > y1:
        return "q2"
    # QUADRANT 3
    elif x > x1 and y > y1:
        return "q3"
    # QUADRANT 4
    elif x > x1 and y < y1:
        return "q4"

def excel_col(col):
    quot, rem = divmod(col-1,26)
    return excel_col(quot) + chr(rem+ord("A")) if col!=0 else ""

def recursive_swing_pos():
    
    g_pos = 180
    d_pos_rem = 0
    g_pos_stack = (0,0,0,0,0)
    
    for k in graph_coords:
        

def recursive_swing(toss,start):
    
    g = 180
    d_rem = 0
    stack = (0,0,0,0,0)
    
    for i in graph[(start[0],start[1])]:
        # make sure radius of swing long enough to catch a post and that it doesn't catch the origin
        if start[2] > i[2] and (start[0],start[1]) != (i[0],i[1]):
            g_2 = calc_angle((toss[0],toss[1]),(start[0],start[1]),(i[0],i[1]))
            if g_2 < g: 
                g = g_2
                d_rem = start[2] - i[2]
                stack = (i[0],i[1],d_rem,i[3],g_2)
                
    # find all the lps
    if stack != (0,0,0,0,0): # and inbounds
        lps = lattice_intercepts((stack[0],stack[1]),stack[2]) 
        if len(lps) > 0:
            for l in lps:
                x = l[0]
                y = l[1]

                g_bound = calc_angle((toss[0],toss[1]),(start[0],start[1]),(stack[0],stack[1]))
                g_swing = calc_angle((toss[0],toss[1]),(start[0],start[1]),(x,y))
                print("angle: "+str(180-abs(g5)))
                print("test: "+str(g_test))
#                 if  0 < g_swing < 180-abs(g_bound):
                ax.plot(x,y,'mo')
                    
        # see if there is more slack...
        if stack[2] > 0:
            print("len remain: "+str(stack[2]))
            recursive_swing(start,stack) 
    else:
        return []
    

IndentationError: expected an indented block (<ipython-input-19-c4228303326f>, line 154)

In [10]:
coords = [(1.0, 1.0), (3.0, 20.0), (5.0, 20.0), (11.0, 20.0), (15.0, 20.0), (17.0, 20.0), (20.0, 20.0), (18.0, 19.0), (19.0, 19.0), (8.0, 18.0), (14.0, 18.0), (16.0, 18.0), (19.0, 18.0), (1.0, 17.0), (9.0, 17.0), (15.0, 17.0), (20.0, 17.0), (4.0, 16.0), (5.0, 16.0), (12.0, 16.0), (18.0, 16.0), (16.0, 15.0), (20.0, 15.0), (13.0, 14.0), (8.0, 13.0), (16.0, 13.0), (19.0, 13.0), (20.0, 12.0), (1.0, 11.0), (4.0, 11.0), (16.0, 9.0), (5.0, 8.0), (8.0, 8.0), (12.0, 7.0), (18.0, 7.0), (16.0, 6.0), (14.0, 5.0), (20.0, 5.0), (12.0, 4.0), (17.0, 4.0), (10.0, 3.0), (8.0, 2.0), (15.0, 2.0), (1.0, 20.0), (6.0, 1.0), (20.0, 1.0)]
graph = {} 


# build the graph
for i in range(len(coords)):
    x1 = coords[i][0]
    y1 = coords[i][1]
    # first toss START
    # x1 = 0
    # y1 = 0
    viable_toss_stack = {}
    toss = []
    toss_length = 0
    for j in coords:
        x2 = j[0]
        y2 = j[1]
        distance = calc_distance(x1,y1,x2,y2)
        slope = calc_slope(x1,y1,x2,y2)
        
        
        
        if slope not in viable_toss_stack:
            viable_toss_stack[slope] = (x2,y2,distance,slope)
        else:
            if abs(viable_toss_stack[slope][2]) > abs(distance):
                del viable_toss_stack[slope]
                viable_toss_stack[slope] = (x2,y2,distance,slope)
            elif abs(viable_toss_stack[slope][2]) == abs(distance):
                viable_toss_stack[slope] = (x2,y2,distance,slope)      

                
    for k in viable_toss_stack:
        toss.append((viable_toss_stack[k][0], viable_toss_stack[k][1], viable_toss_stack[k][2], viable_toss_stack[k][3]))
        # toss.append((viable_toss_stack[k][0], viable_toss_stack[k][1]))
        graph[(x1,y1)] = toss
        

print(graph[(14.0,5.0)])

[(1.0, 1.0, 13.6015, '0.3077_q3'), (3.0, 20.0, 18.6011, '-1.3636_q4'), (5.0, 20.0, 17.4929, '-1.6667_q4'), (11.0, 20.0, 15.2971, '-5.0_q4'), (15.0, 20.0, 15.0333, '15.0_q1'), (20.0, 20.0, 16.1555, '2.5_q1'), (18.0, 19.0, 14.5602, '3.5_q1'), (19.0, 19.0, 14.8661, '2.8_q1'), (8.0, 18.0, 14.3178, '-2.1667_q4'), (14.0, 18.0, 13.0, 'vertical_pos'), (16.0, 18.0, 13.1529, '6.5_q1'), (19.0, 18.0, 13.9284, '2.6_q1'), (1.0, 17.0, 17.6918, '-0.9231_q4'), (9.0, 17.0, 13.0, '-2.4_q4'), (15.0, 17.0, 12.0416, '12.0_q1'), (4.0, 16.0, 14.8661, '-1.1_q4'), (5.0, 16.0, 14.2127, '-1.2222_q4'), (12.0, 16.0, 11.1803, '-5.5_q4'), (18.0, 16.0, 11.7047, '2.75_q1'), (16.0, 15.0, 10.198, '5.0_q1'), (20.0, 15.0, 11.6619, '1.6667_q1'), (13.0, 14.0, 9.0554, '-9.0_q4'), (8.0, 13.0, 10.0, '-1.3333_q4'), (16.0, 13.0, 8.2462, '4.0_q1'), (19.0, 13.0, 9.434, '1.6_q1'), (20.0, 12.0, 9.2195, '1.1667_q1'), (1.0, 11.0, 14.3178, '-0.4615_q4'), (4.0, 11.0, 11.6619, '-0.6_q4'), (16.0, 9.0, 4.4721, '2.0_q1'), (5.0, 8.0, 9.4868, 

In [14]:
print(str(lattice_intercepts((4.0,11.0),calc_distance(4.0,11.0,5,1))))

[(3, 1), (3, 21), (5, 1), (5, 21), (14, 10), (14, 12)]


In [22]:
print(calc_distance(4,10,5,1))
print(calc_distance(4,10,8,2))

print(calc_distance(4,10,5,1)-calc_distance(4,10,8,2))

9.0554
8.9443
0.11110000000000042


In [18]:
print(calc_angle((5,1),(4,11),(2,8)))

-39.4007
